<a href="https://colab.research.google.com/github/andresalcaraaz/Netflix-Project-2-ETL-Data/blob/master/Sonya_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.1.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://ppa.launchpad.net/grap

In [10]:
# Start Spark session
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import StructType,StructField,StringType, DateType,IntegerType

# we are going to use this to time our queries.
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()


In [11]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-netflix/filterNames.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("filterNames.csv"), sep=",", header=True)

# Show DataFrame
df.createOrReplaceTempView('names')

In [12]:
#spark.sql("SELECT * FROM names WHERE primaryprofession = 'actor' OR primaryprofession = 'actress' ").show()
df = spark.sql("SELECT * FROM names WHERE primaryprofession in ('actor', 'actress') ")
clean_professions_df = df.drop("birthyear","knownfortitles","knownFor","deathyear")
clean_professions_df.show()

+---------+--------------------+-----------------+
|   nconst|         primaryname|primaryprofession|
+---------+--------------------+-----------------+
|nm0458601|          Mark Kleid|            actor|
|nm0458664|          Bart Klein|            actor|
|nm0458866|        Leslie Klein|          actress|
|nm0458929|         Paula Klein|          actress|
|nm0459041|     Kevin Kleinberg|            actor|
|nm0459062| Adelheid Kleineidam|          actress|
|nm0459073|      Sergio Kleiner|            actor|
|nm0459084|  Franziska Kleinert|          actress|
|nm0459106|   Stefan Kleinkrieg|            actor|
|nm0459132|    Manny Kleinmuntz|            actor|
|nm0459186|      Maria Klejdysz|          actress|
|nm0459299|     Ladislav Klepal|            actor|
|nm0459318|      Kamaljeet Kler|          actress|
|nm0459364|        Aaron Kleven|            actor|
|nm0459522|      Nele Savicenko|          actress|
|nm0459580|        Atila Klince|            actor|
|nm0459683|       Kent Klineman

In [36]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-netflix/filteredPrincipals.csv"
spark.sparkContext.addFile(url)
df1 = spark.read.csv(SparkFiles.get("filteredPrincipals.csv"), sep=",", header=True)

# Show DataFrame
df1.createOrReplaceTempView('names1')

In [37]:
character.show()

+----------+--------+---------+---------------+----+-----------------+
|    tconst|ordering|   nconst|       category| job|       characters|
+----------+--------+---------+---------------+----+-----------------+
|tt12189034|       6|nm5943320|          actor|null|      "[""Lee""]"|
|tt12189034|       6|nm5943320|          actor|null|      "[""Lee""]"|
|tt12189034|       6|nm5943320|          actor|null|      "[""Lee""]"|
|tt12189034|       6|nm5943320|          actor|null|      "[""Lee""]"|
|tt12189034|       8|nm7672770|          actor|null|"[""Detective""]"|
|tt12189034|       8|nm7672770|          actor|null|"[""Detective""]"|
|tt12189034|       8|nm7672770|          actor|null|"[""Detective""]"|
|tt12189034|       8|nm7672770|          actor|null|"[""Detective""]"|
|tt12190374|       3|nm0767339|cinematographer|null|             null|
|tt12190374|       3|nm0767339|cinematographer|null|             null|
|tt12190374|       3|nm0767339|cinematographer|null|             null|
|tt121

In [39]:
#spark.sql("SELECT * FROM names WHERE primaryprofession = 'actor' OR primaryprofession = 'actress' ").show()
df1 = spark.sql("SELECT * FROM names1 WHERE category in ('actor', 'actress') ")
category = df.drop("tconst","ordering","job")
category.show()

+---------+--------------------+---------+---------+-----------------+--------------------+---------+
|   nconst|         primaryname|birthyear|deathyear|primaryprofession|      knownfortitles| knownFor|
+---------+--------------------+---------+---------+-----------------+--------------------+---------+
|nm0458601|          Mark Kleid|     null|     null|            actor|tt0105041,tt01120...|tt0277615|
|nm0458664|          Bart Klein|     null|     null|            actor|tt0830854,tt03064...|tt0306413|
|nm0458866|        Leslie Klein|     null|     null|          actress|           tt0462601|tt0462601|
|nm0458929|         Paula Klein|     null|     null|          actress|tt1060234,tt01174...|tt1258120|
|nm0459041|     Kevin Kleinberg|     1982|     null|            actor|tt0277249,tt57770...|tt0256278|
|nm0459062| Adelheid Kleineidam|     1966|     null|          actress|tt0178142,tt04372...|tt0437294|
|nm0459073|      Sergio Kleiner|     1936|     null|            actor|tt0495921,tt

In [ ]:
#clean_professions_df[['nconst','category']].merge(clean_characters_df[['characters','primaryname']], on='nconst')
#df[['col1', 'col2']].merge(df2[['col1', 'col3']], on='col1')
rdd1 = sc.parallelize(clean_professions_df)
rdd2 = sc.parallelize(clean_characters_df)

union_rdd = sc.union([rdd1, rdd2])
#new_professions_df=clean_characters_df.join(clean_professions_df,"nconst")